In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os,sys
import errno
import string
from nltk.corpus import reuters 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score,accuracy_score,roc_auc_score,mean_absolute_error
from sklearn.preprocessing import label_binarize
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestCentroid
import csv
from sklearn import svm
import warnings
import random
from statistics import mode
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
random.seed(0)
warnings.filterwarnings('ignore')
def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)
def Cosine(a, b):#distance
    return distance.cosine(a,b)
def EISC(doc1,doc2): 
    dot=np.sum(np.sqrt(np.multiply(doc1,doc2)))
    isc=dot /( math.sqrt(np.linalg.norm(doc1, ord=1))*math.sqrt(np.linalg.norm(doc2, ord=1)))
    return 1-isc
def most_common(lst):
    return(mode(lst))


def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[5,15,30,45],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    if std==[]:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["k",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=train_time+test_time
    for i in range(0,len(Arr)):
        print(i,len(Arr))
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if std==[]:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if std==[]:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)

def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    averageMeanPrecison=[]
    kList=[5,15,30,45]
    func=globals()[metric]
    train_time=[]
    test_time=[]
    maxval=np.max(kList)
    var=np.zeros(train_data.shape[1])
    class_dist=[]
    label_dist=[]
    ##########Main piece of code############
    # split training data according to class distribution
    for k in kList:
        time1=time.time()
        pred=[]
        print("k",k)
        k1=k
        if k1<15:
            k1=k*(len(classes)+1) 
        classifier = KNeighborsClassifier(n_neighbors=k1,metric=func)
        if len(train_labels)<k1:
            k=len(train_labels)
            classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
        classifier.fit(train_data,train_labels)
        distances,neighs=classifier.kneighbors(test_data,return_distance=True)
        distances=distances

        time2=time.time()
        for t in range(0,test_data.shape[0]):
            weights2=[]
            dist=distances[t,:]
            neighs2=neighs[t,:]
            weights2=np.ones((len(dist)))
            if (dist[-1]-dist[0]) !=0:
                weights2 = (dist[-1] - dist)/(dist[-1]-dist[0])
            neigh_data=train_data[neighs[t],:]
            neigh_dist=distances[t,:]
            neigh_labels=[train_labels[n] for n in neighs2]
            weightSum=0
            maxweight=-np.inf
            predLabel=''
            for lbl in set(neigh_labels):
                w3=np.nansum([weights2[j]/(distances[t,j]+0.00001) for j,ii in enumerate(neighs[t]) if train_labels[ii]==lbl])
                weightSum=w3
                if weightSum>=maxweight:
                    maxweight=weightSum
                    predLabel=lbl
            pred.append(predLabel)
        y_pred.append(pred) 
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
          
    class_dict={}
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)

        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)

    #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Latest Datasets\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Latest Datasets\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    #pca=PCA(n_components=3)
    #arr=pca.fit_transform(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = preprocessing.LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    classes=list(set(labels))
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "EDWKNN\\April Task1\\Latest Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_EDWKNN.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=xTrain
            test_data=xTest
            train_labels=yTrain
            test_labels=yTest
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





[nltk_data] Downloading package reuters to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


b'Latest Datasets\\banknote'
###############
k 5
k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+-----------------------+---------------------+
|    k     | Accuracy |      Train Time      |       Test time       |      Total Time     |
+----------+----------+----------------------+-----------------------+---------------------+
|    5     |  0.9565  |    0:00:00.569276    |     0:00:00.008033    |    0:00:00.569276   |
|    15    |  0.9565  |    0:00:00.598247    |     0:00:00.019728    |    0:00:00.598247   |
|    30    |  0.9493  |    0:00:00.636172    |     0:00:00.015620    |    0:00:00.636172   |
|    45    |  0.9493  |    0:00:00.613034    |        0:00:00        |    0:00:00.613034   |
| Average  |  0.9529  |    0:00:00.604182    |     0:00:00.010845    |    0:00:00.307514   |
| Std Dev. |  0.0036  | 0.024266272764155213 | 0.0075372496096401155 | 0.29721212536619773 |
+----------+----------+-------------

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+---------------------+----------------------+--------------------+
|    k     | Accuracy |      Train Time     |      Test time       |     Total Time     |
+----------+----------+---------------------+----------------------+--------------------+
|    5     |  0.942   |    0:00:00.503742   |    0:00:00.015621    |   0:00:00.503742   |
|    15    |  0.942   |    0:00:00.539463   |       0:00:00        |   0:00:00.539463   |
|    30    |  0.9493  |    0:00:00.562372   |    0:00:00.015618    |   0:00:00.562372   |
|    45    |  0.9493  |    0:00:00.691035   |       0:00:00        |   0:00:00.691035   |
| Average  |  0.9457  |    0:00:00.574153   |    0:00:00.007810    |   0:00:00.290981   |
| Std Dev. |  0.0036  | 0.07064225427557991 | 0.007809698684050412 | 0.2875965830680771 |
+----------+----------+---------------------+----------------------+--------------------+
metric Euclidean
measu

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+---------------------+----------------------+--------------------+
|    k     | Accuracy |      Train Time     |      Test time       |     Total Time     |
+----------+----------+---------------------+----------------------+--------------------+
|    5     |  0.9416  |    0:00:00.546747   |    0:00:00.015623    |   0:00:00.546747   |
|    15    |  0.9416  |    0:00:00.487988   |    0:00:00.015622    |   0:00:00.487988   |
|    30    |  0.9343  |    0:00:00.562369   |    0:00:00.015618    |   0:00:00.562369   |
|    45    |  0.9197  |    0:00:00.691116   |       0:00:00        |   0:00:00.691116   |
| Average  |  0.9343  |    0:00:00.572055   |    0:00:00.011716    |   0:00:00.291885   |
| Std Dev. |  0.0089  | 0.07412328061696156 | 0.006764068687443274 | 0.2850701636809795 |
+----------+----------+---------------------+----------------------+--------------------+
metric Euclidean
measu

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+---------------------+----------------------+---------------------+
|    k     | Accuracy |      Train Time     |      Test time       |      Total Time     |
+----------+----------+---------------------+----------------------+---------------------+
|    5     |  0.9708  |    0:00:00.519422   |       0:00:00        |    0:00:00.519422   |
|    15    |  0.9708  |    0:00:00.484261   |    0:00:00.015622    |    0:00:00.484261   |
|    30    |  0.9708  |    0:00:00.554026   |    0:00:00.015622    |    0:00:00.554026   |
|    45    |  0.9708  |    0:00:00.593610   |    0:00:00.015621    |    0:00:00.593610   |
| Average  |  0.9708  |    0:00:00.537830   |    0:00:00.011716    |    0:00:00.274773   |
| Std Dev. |   0.0    | 0.04056544839466248 | 0.006764515791376539 | 0.26465906266004113 |
+----------+----------+---------------------+----------------------+---------------------+
metric Eucli

...

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+-----------+---------------------+
|    k     | Accuracy |      Train Time      | Test time |      Total Time     |
+----------+----------+----------------------+-----------+---------------------+
|    5     |   0.75   |    0:00:00.031243    |  0:00:00  |    0:00:00.031243   |
|    15    |   0.75   |    0:00:00.015590    |  0:00:00  |    0:00:00.015590   |
|    30    |   0.75   |    0:00:00.031268    |  0:00:00  |    0:00:00.031268   |
|    45    |   0.75   |    0:00:00.015621    |  0:00:00  |    0:00:00.015621   |
| Average  |   0.75   |    0:00:00.023431    |  0:00:00  |    0:00:00.011715   |
| Std Dev. |   0.0    | 0.007825148745406399 |    0.0    | 0.01295630971646652 |
+----------+----------+----------------------+-----------+---------------------+
metric Euclidean
measure Precision
0 4
1 4
2 4
3 4
+----------+-----------+----------------------+-----------+--

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+-----------------------+-----------+---------------------+
|    k     | Accuracy |       Train Time      | Test time |      Total Time     |
+----------+----------+-----------------------+-----------+---------------------+
|    5     |   0.6    |     0:00:00.015621    |  0:00:00  |    0:00:00.015621   |
|    15    |   0.6    |     0:00:00.031243    |  0:00:00  |    0:00:00.031243   |
|    30    |   0.7    |     0:00:00.015621    |  0:00:00  |    0:00:00.015621   |
|    45    |   0.75   |     0:00:00.015622    |  0:00:00  |    0:00:00.015622   |
| Average  |  0.6625  |     0:00:00.019527    |  0:00:00  |    0:00:00.009763   |
| Std Dev. |  0.065   | 0.0067642749181795825 |    0.0    | 0.01087194374139149 |
+----------+----------+-----------------------+-----------+---------------------+
metric Euclidean
measure Precision
0 4
1 4
2 4
3 4
+----------+-----------+-----------------------+---

metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+------------------------+----------------------+-----------------------+
|    k     | Accuracy |       Train Time       |      Test time       |       Total Time      |
+----------+----------+------------------------+----------------------+-----------------------+
|    5     |   0.75   |     0:00:00.015621     |       0:00:00        |     0:00:00.015621    |
|    15    |   0.75   |     0:00:00.015621     |    0:00:00.015626    |     0:00:00.015621    |
|    30    |   0.75   |     0:00:00.015627     |       0:00:00        |     0:00:00.015627    |
|    45    |   0.75   |     0:00:00.015618     |       0:00:00        |     0:00:00.015618    |
| Average  |   0.75   |     0:00:00.015622     |    0:00:00.003907    |     0:00:00.009764    |
| Std Dev. |   0.0    | 3.2488632380751714e-06 | 0.006766339658431517 | 0.0075633130026891695 |
+----------+----------+------------------------+----------------------+---------------

metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+-----------+----------------------+
|    k     | Accuracy |      Train Time      | Test time |      Total Time      |
+----------+----------+----------------------+-----------+----------------------+
|    5     |   0.4    |    0:00:00.031248    |  0:00:00  |    0:00:00.031248    |
|    15    |   0.4    |    0:00:00.015621    |  0:00:00  |    0:00:00.015621    |
|    30    |   0.7    |    0:00:00.015622    |  0:00:00  |    0:00:00.015622    |
|    45    |   0.7    |    0:00:00.031245    |  0:00:00  |    0:00:00.031245    |
| Average  |   0.55   |    0:00:00.023434    |  0:00:00  |    0:00:00.011717    |
| Std Dev. |   0.15   | 0.007812678892603817 |    0.0    | 0.012954082991381346 |
+----------+----------+----------------------+-----------+----------------------+
metric Euclidean
measure Precision
0 4
1 4
2 4
3 4
+----------+-----------+----------------------+-----------+--------------------

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 4
1 4
2 4
3 4
+----------+----------+----------------------+----------------------+----------------------+
|    k     | Accuracy |      Train Time      |      Test time       |      Total Time      |
+----------+----------+----------------------+----------------------+----------------------+
|    5     |  0.3158  |    0:00:00.015621    |       0:00:00        |    0:00:00.015621    |
|    15    |  0.3158  |    0:00:00.031212    |       0:00:00        |    0:00:00.031212    |
|    30    |  0.6316  |    0:00:00.015621    |       0:00:00        |    0:00:00.015621    |
|    45    |  0.7895  |    0:00:00.015647    |    0:00:00.015647    |    0:00:00.015647    |
| Average  |  0.5132  |    0:00:00.019525    |    0:00:00.003912    |    0:00:00.011718    |
| Std Dev. |  0.2051  | 0.006747317976152628 | 0.006775424626465907 | 0.010327667957582483 |
+----------+----------+----------------------+----------------------+----------------------+
metric 

In [2]:
test=[1,2,3,4]
print(test[0:-1])

[1, 2, 3]


In [3]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
y_scores = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
roc_auc_score(y_true, y_scores)

0.5